### 1. 分词器的概念
分词器(Tokenizer)是将自然语言文本转换为模型可理解形式的工具，它将连续的文本字符串分解 成模型能够处理的基本单位。这些基本单位可以是单词、子词(subword)，甚至是单个字符。分词的 目的是将原始文本映射为一系列的索引或标记(token)，然后输入到模型中进行处理。
  分词器的工作不仅限于切割文本，还包括将这些分割后的单位映射到模型词汇表中的索引。分词的
粒度选择会影响模型的性能、训练时间和生成的结果，因此分词器在训练过程中扮演着重要角色。

### 2. 为什么需要训练分词器? 在大模型训练过程中，分词器是模型理解和生成文本的基础。如果没有合理的分词，模型无法有效地学习语言的规律和结构。以下是训练分词器的主要原因:
#### 2.1 处理文本的必要性
计算机无法直接处理自然语言文本，需要将其转换为数字表示。而分词器就是完成这一步的工具，它将文本转换为一系列离散的标记，之后可以通过嵌入层将这些标记转换为向量输入模型。因此，训练分词器的首要任务是高效且准确地文本分解成适合模型学习的标记。
#### 2.2 减少模型的复杂性
如果直接使用单词作为标记，模型词汇表将变得极其庞大，因为自然语言中有海量的单词及其变体。词汇表过大不仅会导致训练时间增加，还会让模型难以泛化，尤其是当遇到词汇表中未包含的新词时，模型将无法处理。通过训练分词器，尤其是使用子词分词(例如BPE或WordPiece算法)，可以将词汇表的大小控制在 一个合理的范围内，同时确保模型能够有效处理新词和变体。子词分词的思想是，将常见的词作为整体 保留，但对不常见的词进行进一步分解，这样即使是未见过的词，也可以通过子词组合来表示。
#### 2.3 提升模型的泛化能力
一个好的分词器能够帮助模型在面对未知或罕见词汇时保持良好的表现。训练好的分词器能够根据训练语料库中的频率、上下文等信息，对词语进行合理的拆分，让模型在推理和生成时具备更强的泛化能力。例如，遇到罕见或未见过的词时，分词器可以将其分解为多个子词，这样即使模型没有见过完整的单词，也能通过学习到的子词来理解其含义。
#### 2.4 提高模型的效率
分词器还可以显著提高模型的计算效率。较小的词汇表可以减少嵌入矩阵的大小，从而减少模型的参数量和训练时间。此外，通过子词分解，还可以避免对整个单词进行处理，提高模型处理长文本时的效率。
### 3. 分词器的作用
#### 3.1 将文本转换为标记序列
分词器的首要作用是将连续的自然语言文本转换为一系列的标记序列。模型需要处理的是这些离散的标记，而不是原始的字符串。不同的分词策略(如按单词、按子词、按字符等)会影响标记的数量和粒度。选择合适的分词策略对于提高模型的表现非常重要。
#### 3.2 构建词汇表
分词器的另一个作用是基于训练语料库构建一个有限大小的词汇表。这个词汇表包含了模型能够处理的所有标记的集合，分词器在训练时通过分析语料库中的词频和词形变化，选择最适合的子词或词汇构建词汇表，确保词汇表足够小但仍能覆盖大部分文本。
#### 3.3 处理未登录词(OOV问题)
自然语言的词汇是动态且丰富的，在训练数据中可能无法包含所有的词汇。分词器通过将未登录词 (Out-Of-Vocabulary，OOV)拆解为子词，能够有效解决OOV问题。例如，对于一个未见过的复杂词汇，分词器可以将其拆分为更常见的子词，从而保证模型可以处理这些新词，而不会因为未登录词的出 现导致模型无法理解。
#### 3.4 保持文本的语义一致性
一个好的分词器不仅仅是将文本简单拆解，还需要考虑语义上的一致性。例如，分词器应该尽量避免将具有特定语义的词错误地分割，以免影响模型的语义理解能力。同时，分词器应通过合理的标记组合，使得模型能够在不同的上下文中准确理解词语的含义。

# step 1. 先导入必要的库

In [1]:
import random
from tqdm import tqdm
from transformers import AutoTokenizer
import json
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import os
import time

/opt/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# step 2. 读取tokenizer_train.jsonl文件

In [2]:
def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

# 测试读取数据
data_path = '../data/raw/tokenizer_train.jsonl'
texts = read_texts_from_jsonl(data_path)

# 打印前几行
for i, text in enumerate(texts):
    if i < 5:
        print(text)
    else:
        break

好的。现在请你将这个文本中的所有的逗号都替换成空格。 好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？
帮我回答一道历史题目。清朝时期的八旗共有多少旗人？ 清朝时期八旗旗人总数约为200万人左右，其中正黄旗、正蓝旗、正白旗、正红旗的人数较多，其他旗的人数较少。
嗯，谢谢你介绍的做法很详细，但我不喜欢吃鸡蛋，有没有其他菜做法能介绍一下？ 当然，你可以试试酸辣土豆丝这道菜。
材料：
土豆2个、红椒1个、青椒1个、大葱1根、醋、生抽、盐、鸡精、料酒
做法：
1.土豆去皮，切成丝；红椒和青椒切成细丝；大葱切段备用。
2.热锅凉油，油热后放入土豆丝，煸炒至变软。
3.倒入红椒、青椒和大葱段，继续煸炒至熟。
4.加入适量的盐、鸡精、料酒和生抽，翻炒均匀。
5.最后，加入适量的醋，翻炒均匀即可。
小贴士：
1. 土豆切丝时，可以放入淡盐水中泡一下，这样可以去除多余的淀粉。
2. 煮土豆丝时，不要煮得太久，以免烂糊。
3. 加入醋的时候，根据自己的口味多少来进行调节，一般来说，盐与醋的比例为1:1。
4. 如果喜欢辣味可以加入一些干辣椒丝。
希望你会喜欢这道酸辣土豆丝！
请描述一下如何正确规划个人理财。 正确规划个人理财需要以下几个步骤：
1.了解自己的财务状况。这包括收入、支出、资产和负债等信息。了解自己的财务状况可以帮助人们更好地制定财务计划。
2.设定财务目标。需要考虑短期目标和长期目标，例如以年为单位设定的支出计划、购房、购车等的长期目标。
3.制定预算计划。在了解自己的财务状况并设定财务目标后，需要制定一个预算计划。这可以帮助人们控制支出、节省开支并达到财务目标。
4.理性投资和储蓄。人们可以投资于股票、基金、房产或其他投资渠道以实现财务目标。但在投资前需了解相关知识并进行风险评估。同时还应储蓄一定金额，以应对突发事件或为达成某些目标做准备。
5.审时度势，合理调整。财务计划需要不断地审时度势，根据实际情况做出调整，以达到最终的财务目标。需要注意财务状况的变化、投资的收益和风险等因素。
通过以上五个步骤，人们可以做到合理规划个人理财，掌握自己的财务命运，更好地实现自己的财务目标。
描述一下天堂和地狱的生态系统和环境。 天堂和地狱被认为是灵性信仰中关于死后世界的两种不同概

# step 3. 初始化分词器

首先，通过 `models.BPE()` 创建了一个基于 Byte-Pair Encoding (BPE) 模型的分词器。BPE 是一种常用于文本分词的子词分解算法，特别在自然语言处理任务中被广泛使用，如机器翻译和语言模型训练。BPE 的主要思想是通过将频繁出现的字符或字符对合并成一个新的子词单元，逐步构建一个子词级别的词汇表，从而处理词汇表稀疏性和未登录词问题。

In [3]:
# 初始化tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# 定义特殊token
special_tokens = ["<unk>", "<s>", "</s>"]

# 设置训练器并添加特殊token
trainer = trainers.BpeTrainer(
    vocab_size = 6400,
    special_tokens = special_tokens,  # 确保这三个token被包含
    show_progress = True,
    initial_alphabet = pre_tokenizers.ByteLevel.alphabet()
)

print("分词器初始化成功，准备训练...")

分词器初始化成功，准备训练...


# step 4. 训练分词器

In [4]:
# 读取文本
texts = read_texts_from_jsonl(data_path)

start_time = time.time()
# 训练tokenizer
tokenizer.train_from_iterator(texts, trainer=trainer)

end_time = time.time()

print(f"分词器训练完成，训练时间为：{round((end_time - start_time)/60,2)} 分钟")




分词器训练完成，训练时间为：41.13 分钟


# step 5. 保存分词器

In [6]:
# 设置解码器
tokenizer.decoder = decoders.ByteLevel()

# 保存tokenizer
tokenizer_dir = "../models/tokenizer_model"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save("../models/tokenizer_model")

# 手动配置文件
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": True,
    "added_token_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rsstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 1000000000000000019884624838656,
    "pad_token": None,
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "use_default_system_prompt": False,
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<s>user\n' + content + '</s>\n<s>assistant\n' }}{% elif message['role'] == 'assistant' %}{{ content + '</s>' + '\n' }}{% endif %}{% endfor %}"
}

# 保存配置文件
with open(os.path.join(tokenizer_dir, "tokenizer_config.json"),"w", encoding='utf-8') as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer 保存成功！")

Tokenizer 保存成功！
